## 21.01.09

- **validation set을 새로 지정하기**
    - 기존에는 KFold / train_test_split의 random_seed = 42를 통해 train / validation set 지정
        - 이런식으로 하니, **validation과 test 점수간 괴리가 너무 큼**
        - test data를 살펴보면 여름 ~ 겨울의 데이터가 주인 듯 (RH)
        - 시계열 데이터의 경우, **가장 최신의 데이터를 validation set으로** 두는 것이 일반적
    - ∴ 가장 최신 1년 데이터(Day 730 ~ 1095)중 여름 ~ 겨울 데이터의 일부를 validation set으로 지정
        - But, Dacon에 따르면 test 데이터는 온전히 추론용으로만 사용해야만 함
        - ∴ 직접 데이터를 골라내는 것이 아니라 **1095X0.2=219일(=7개월정도)만큼의 최신 데이터를 validation set으로 지정**
- **해볼만한 시도들 목록**
    - **1.**계절 및 몇 월인지 등 **날짜에 관한 feature** 생성
        - 계절, 몇월인지 등을 이용하면 시계열 데이터에서 더욱 효과적으로 발휘할 듯
        - 실제로 4위팀의 팀원이 Month를 예측하는 모델을 사용해서 test 데이터에 feature를 추가해도 되냐는 질문을 함
    - **2.** 범위가 크고 skewed된 수치형 데이터의 경우 **log transformation**
    - **3.** **시간(00:00 ~ 23:30)별 & 예측 날짜(1일후, 2일후)별 & quantile 별**로 각각 모델 생성  
        - 제품을 예측하는 시계열 관련 캐글 우승자의 경우, 제품별로 다른 피처 엔지니어링/모델튜닝을 수행하여 조합하였다고 함
        - 마찬가지로 **시간(00:00 ~ 23:30)별 & 예측 날짜(1일후, 2일후)별로 다른 피처 엔지니어링/모델튜닝을 수행**하면 좋을 듯
    - **4.** test 데이터와 닮은 train data만 이용해서 모델 학습
        - 근데, Dacon에서 test data를 분석하는 것을 막았으니.. 왜 막았는지 이해가 안된다...
    - **5.** **상관관계**가 높은 완전히 변수 삭제 (완전히 다른 성질의 변수들 간)
        - 시계열 데이터에서도 먹힐지는 의문
    - **6.** 타겟변수가 안

#### 머신러닝 책에서 얻은 정보들
- **About Model**
    - XGBoost를 많이 사용하는 것 같음. 
        - parameter: max_depth, eta, colsample_bytree(0.6 ~ 0.9), colsample_bylevel(0.6 ~ 0.9)
    - 인공신경망의 경우, -1 ~ 1 사이의 값을 입력값으로 받을 때 가장 효과적
- **About validation & test model**
    - validation set을 통해 어느정도 model의 파라미터를 조정한 후, **validation set까지도 이용해 model을 다시 training**
    - 이때, **늘어난 데이터의 비율만큼 트리의 개수(n_estimators) 역시 증가시킴** 
        - early stopping을 이용한 경우, XGBoost: **model.best_ntree_limit** & LGBM: **model.best_iteration** 이용
- **시계열 데이터**
    - 시계열 데이터의 경우 **lag-n의 데이터**가 상당한 설명력을 부여 (해당 feature의 n시점 전의 데이터)
    - n의 경우 여러 실험을 통해 정해야 함. task에 따라 적절한 n이 다름
    - 또한, **lag-n 데이터의 기초통계량**(평균, 표준편차, 최댓값, 최솟값)을 새로운 feature로 사용 가능
        - lag-n 데이터 자체보다 구간을 정해서 **구간별 표준편차, 최댓값, 최솟값**을 이용하는 것도 좋은 듯 (EX. if n=5, [1-3],[1-5],[2-5],[2-3],[2-5])
    
    

---

## 21.01.10

- **오늘 해볼 것들 목록 **
    1. DHI, DNI, TARGET: 각각의 변수에 대해 파생변수 2개 (0인지 아닌지 & 0이 아닌 경우 해당 값들의 log 정규화 생성
        - 0에 해당하는 데이터가 두 feature 모두 50%를 넘음
        - DHI의 경우 DNI가 0인지 아닌지에 따라 Target에 끼치는 영향력 자체가 달라짐 → cloudy에 해당하는 feature 대체 가능
        - 이때, Target은 예측하는 변수로써가 아닌 파생변수로써의 Target
    2. Validation Set을 새롭게 지정한 후, 기존의 parameter setting에서 validation 값이 어느정도 나오는지 확인
        - 아래 칸에 기입 
    3. 시간(00:00 ~ 23:00)별 & 예측 날짜(1일 후, 2일후)별 데이터 생성
        - 문제는 이럴 경우 test data의 04:00나 19:30과 같이 train data에는 나타나지 않는 경우에 대해 예측 불가능
        - 완전한 시간별 모델 생성 및 예측은 불가능
        - 시간별 Target을 보고 아침/점심/저녁과 같이 구간별로 예측하는 모형 역시 생각해보기
    4. 계절, Month, feature 추가해보기